### **Script base: spazzata in frequenza**

Il notebook mostra come fare una scansione in frequenza per la misura del diagramma di Bode. Lo script si basa 
non su una FFT ma semmai su un fit con un coseno.

Parametri chiave

* `nper` imposta il numero di periodi misurati nell'acquisizione... dato il metodo di stima (fit), ne basta uno
* `nf` imposta il numero di frequenze nello scan
* `npt` imposta il numero dei punti da acquisire
* `f0` imposta la frequenza iniziale
* `f1` imposta la frequenza finale
* `flag_return` => attiva andata e ritorno
    
> **Come è scelto il campionamento?**. Lo script cerca di acquisire `nper` periodi con `npt` punti che, dato un segnale a frequenza `ff` in teoria richiede a
>
> ```python
>        .fc = ff * npt / nper
> ```
>
>tuttavia, $\texttt{Analog Discovery 2}$ può solo campionare a sottomultipli $(100/n)\,{\rm MHz}$. Lo script quindi calcola $n$ con la formula di sopra e lo scope.ch1.valsotonda *verso l'alto*, calcolando poi quale sia l'esatto numero di punto per coprire il numero di periodi richiesto. L'scope.ch1.valsotondamento verso l'alto è stato scelto in modo che il numero di punti finale non superi mai quello impostato nei parametri.

> **Attenzione**. La qualità dello studio declina rapidamente sopra i $100\,{\rm kHz}$ ed è certamente sconsigliato andare sopra $1\,{\rm MHz}$. Il motivo più probabile per le discrepanze che si sviluppano è legato al cross-talk fra i cavi del *bundle* fornito, che non è particolarmente adatto alle alte frequenze. 

In [ ]:
import tdwf
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt 
import numpy as np
import scipy.optimize as so
import math
    
# ==========================================================================
#  Parametri dello script
nper = 2            # numero periodi usati per la stima   
npt = 8192          # numero MASSIMO di punti acquisiti
nf = 100            # numero di frequenze nello sweep da f0 a f1   
f0 = 50        
f1 = 500e3      
flag_return = False  # A/R? o solo andata?
flag_show = True     # Visualizza i segnali nel tempo? (è uguale...)
# vettore delle frequenze
fv = np.logspace(np.log10(f0), np.log10(f1), nf) 

# ==========================================================================
#  Configurazione base AD2 (più parametri impostati dopo)
ad2 = tdwf.AD2()
ad2.vdd = 5
ad2.vss = -5
ad2.power(True)

wavegen = tdwf.WaveGen(ad2.hdwf)
wavegen.w1.ampl = 3
wavegen.w1.func = tdwf.funcSine 
wavegen.w1.start()

scope = tdwf.Scope(ad2.hdwf)
scope.ch1.rng = 50
scope.ch2.rng = 50

# ==========================================================================
#   Esecuzione...

# [1] Crea plot
fig, [[ax1, ax3], [ax2, ax4]] = plt.subplots(2, 2, figsize=(12, 6),                                
    gridspec_kw={'width_ratios': [1, 2]})
fig.canvas.manager.set_window_title('Spazzata frequenza')

# [2] Preparazione dataset
flag_first = True
Am = np.full((nf, 2), np.nan)
phim = np.full((nf, 2), np.nan)

# [3] Ciclo misura
for ar in range(2 if flag_return else 1):  # Ciclo A/R
    for ii in range(len(fv)):  # Ciclo frequenze
        # [3a] Calcolo frequenza
        if ar == 0:
            findex = ii
        else:
            findex = len(fv) - ii - 1

        ff = fv[findex]

        # [3b] Stima parametri di sampling
        df = math.ceil(100e6 * nper / (npt * ff))
        scope.fs = 100e6 / df
        scope.npt = int(scope.fs * nper / ff)

        scope.trig(True, hist=0.01)
        wavegen.w1.freq = ff 

        # [3c] Campionamento e analisi risultati
        scope.sample()

        A1 = (np.mean(scope.ch1.vals[max(np.argmax(scope.ch1.vals) - 10, 0)
              : min(np.argmax(scope.ch1.vals) + 11, len(scope.ch1.vals))])
              - np.mean(scope.ch1.vals[max(np.argmin(scope.ch1.vals) - 10, 0)
              : min(np.argmin(scope.ch1.vals) + 11, len(scope.ch1.vals))]))

        A2 = (np.mean(scope.ch2.vals[max(np.argmax(scope.ch2.vals) - 10, 0)
              : min(np.argmax(scope.ch2.vals) + 11, len(scope.ch2.vals))])
              - np.mean(scope.ch2.vals[max(np.argmin(scope.ch2.vals) - 10, 0)
              : min(np.argmin(scope.ch2.vals) + 11, len(scope.ch2.vals))]))

        # [3d] Aggiustamento della fase (se ampiezza negativa...)
        # [3e] Aggiornamento dei dati
        Am[findex, ar] = A2 / A1

        # [3f] Aggiornamento plots
        if flag_first:
            flag_first = False
            if flag_show:
                xx = np.linspace(0, 1000 * scope.time.vals[-1], 1000)

                ymax = A1 / 2 * np.ones_like(xx)
                ymin = -A1 / 2 * np.ones_like(xx)
                ax1.plot(xx, ymax, ":", color="tab:orange")
                ax1.plot(xx, ymin, ":", color="tab:orange")
                
                ymax = A2 / 2 * np.ones_like(xx)
                ymin = -A2 / 2 * np.ones_like(xx)
                ax2.plot(xx, ymax, ":", color="tab:orange")
                ax2.plot(xx, ymin, ":", color="tab:orange")

                hp1, = ax1.plot(1000 * scope.time.vals, scope.ch1.vals, "-", label="Ch1", color="tab:orange")
                hp2, = ax2.plot(1000 * scope.time.vals, scope.ch2.vals, "-", label="Ch2", color="tab:blue")

                ax1.grid(True)
                ax2.grid(True)
                ax1.set_xticks([])
                ax1.set_ylabel("Ch1 [V]", fontsize=15)
                ax2.set_xlabel("Time [msec]", fontsize=15)
                ax2.set_ylabel("Ch2 [V]", fontsize=15)
                ax1.set_xlim([0, nper / ff * 1000])
                ax2.set_xlim([0, nper / ff * 1000])
                ax1.set_title("Starting")

            hp3A, = ax3.loglog(fv, Am[:, 0], ".", markerfacecolor="none", label="Amp go", color="tab:orange")
            hp4A, = ax4.semilogx(fv, phim[:, 0], ".", markerfacecolor="none", label="phi go", color="tab:orange")

            if flag_return:
                hp3R, = ax3.loglog(fv, Am[:, 1], "v", markerfacecolor="none", label="Amp return", color="tab:blue")

            ax3.grid(True)
            ax4.grid(True)            
            ax3.set_xticks([])
            ax3.yaxis.tick_right()
            ax3.yaxis.set_label_position('right')
            ax3.set_ylabel("Gain [pure]", fontsize=15)
            ax4.set_xlabel("Freq [Hz]", fontsize=15)
            ax4.yaxis.tick_right()
            ax4.yaxis.set_label_position('right')
            ax4.set_yticks([-np.pi, -np.pi/2, 0, np.pi/2, np.pi])
            ax4.set_yticklabels(["-180", "-90", "0", "+90", "+180"])
            ax4.set_ylabel("Phase [deg]", fontsize=15)
            ax3.legend()
            ax4.legend()
            plt.tight_layout()
            plt.show(block=False)    
        else:
            # Aggiorna titolo
            if ff > 1e3:
                title = f"{ff / 1e3:.1f} kHz"
            else:
                title = f"{ff:.1f} Hz"

            if scope.fs > 1e6:
                ax1.set_title(f"{title} [{scope.npt:d} pts @ {scope.fs/1e6:.1f} MSa/s]")
            elif scope.fs > 1e3:
                ax1.set_title(f"{title} [{scope.npt:d} pts @ {scope.fs/1e3:.1f} kSa/s]")
            else:
                ax1.set_title(f"{title} [{scope.npt:d} pts @ {scope.fs:.1f} Sa/s]")

            # Aggiorna dati
            if flag_show:
                hp1.set_xdata(1000 * scope.time.vals)
                hp1.set_ydata(scope.ch1.vals)
                hp2.set_xdata(1000 * scope.time.vals)
                hp2.set_ydata(scope.ch2.vals)
                ax1.set_xlim([0, nper / ff * 1000])
                ax2.set_xlim([0, nper / ff * 1000])
                mi = np.min(scope.ch2.vals)
                ma = np.max(scope.ch2.vals)
                dm = ma - mi
                m0 = (ma + mi) / 2
                ax2.set_ylim([m0 - 0.6 * dm, m0 + 0.6 * dm])

            if ar == 0:
                hp3A.set_ydata(Am[:, 0])
                hp4A.set_ydata(phim[:, 0])
                ax3.set_xlim([f0, ff])
                ax3.set_ylim([0.5 * np.nanmin(Am[:, 0]), 2 * np.nanmax(Am[:, 0])])
                ax4.set_xlim([f0, ff])
            else:
                hp3R.set_ydata(Am[:, 1])

            fig.canvas.draw()
            fig.canvas.flush_events()

data = np.column_stack((fv, Am, phim))
# np.savetxt("AD8031_Gain1e3.txt", data)

# ---------------------------------------
ad2.close()


Digilent WaveForms SDK versione 3.22.2
Dispositivo #1 [SN:210321B5DD07, hdwf=1] connesso!
Configurazione #1
Dispositivo disconnesso.


# Salvataggio dei dati...

Ovviamente si potrebbe mettere in un unico script, ma è sano poter cambiare qualcosa qui senza dover runnare da zero la misura...

In [8]:
data = np.column_stack([fv,Am, phim])
np.savetxt("Output.txt", data)